In [1]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

2024-09-22 23:56:49.899580: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-22 23:56:49.972112: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-22 23:56:49.992172: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-22 23:56:50.118694: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-22 23:56:51.319443: W tensorflow/compiler/tf2

In [2]:
from tensorflow.keras import mixed_precision

# Set the global policy to mixed precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)


In [ ]:
import os

# Set the environment variable
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

In [3]:
# data = pd.read_csv("C:\Users\alanx\OneDrive - The University of Sydney (Students)\Thesis\Deep Learning\wifi_localization\data\csv\ultimate_combined.csv")
# data = pd.read_csv("data/csv/ultimate_combined.csv")
data = pd.read_csv("data/csv/still/still.csv")
data.head()

,timestamps,csi_len,channel,err_info,noise_floor,rate,bandWidth,num_tones,nr,nc,...,payload_length,block_length,subcarriers,ant1_amplitude,ant2_amplitude,ant1_phase,ant2_phase,with_receiver,coord1,coord2
0,429192390,560,2437,0,0,140,0,56,2,2,...,1320,1904,1,27.459060,92.070625,0.992894,-1.418147,0,1,1
1,429192390,560,2437,0,0,140,0,56,2,2,...,1320,1904,2,27.459060,167.839209,-0.992894,1.994465,0,1,1
2,429192390,560,2437,0,0,140,0,56,2,2,...,1320,1904,3,28.844410,94.339811,0.982794,-1.485895,0,1,1
3,429192390,560,2437,0,0,140,0,56,2,2,...,1320,1904,4,29.154759,174.642492,-1.030377,1.983207,0,1,1
4,429192390,560,2437,0,0,140,0,56,2,2,...,1320,1904,5,30.805844,89.453899,0.946773,-1.470015,0,1,1


In [4]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


I0000 00:00:1727013436.152976    8688 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727013436.289986    8688 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727013436.290050    8688 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [5]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

['/device:CPU:0', '/device:GPU:0']


I0000 00:00:1727013436.301582    8688 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727013436.301732    8688 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727013436.301769    8688 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727013436.780169    8688 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727013436.780253    8688 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-09-22

In [6]:
variances = data.var()
print(variances)

low_variance_cols = variances[variances < 5e-3].index.tolist()
print("Columns with low or zero variance:")
print(low_variance_cols)

timestamps        1.929837e+18
csi_len           9.590696e+03
channel           2.805950e-03
err_info          4.619301e-03
noise_floor       0.000000e+00
rate              8.232657e+00
bandWidth         0.000000e+00
num_tones         0.000000e+00
nr                0.000000e+00
nc                1.223303e-01
rssi              7.971408e+00
rssi1             4.820231e+00
rssi2             1.001318e+01
rssi3             0.000000e+00
payload_length    1.846202e+04
block_length      2.814382e+04
subcarriers       2.612500e+02
ant1_amplitude    4.187203e+03
ant2_amplitude    5.506470e+03
ant1_phase        3.289260e+00
ant2_phase        3.291396e+00
with_receiver     0.000000e+00
coord1            3.996401e+00
coord2            2.251143e+00
dtype: float64
Columns with low or zero variance:
['channel', 'err_info', 'noise_floor', 'bandWidth', 'num_tones', 'nr', 'rssi3', 'with_receiver']


In [7]:
# Drop constant columns and timeframe from the DataFrame
data.drop(low_variance_cols, axis=1, inplace=True)
data.drop('timestamps', axis=1, inplace=True)
data.head()

,csi_len,rate,nc,rssi,rssi1,rssi2,payload_length,block_length,subcarriers,ant1_amplitude,ant2_amplitude,ant1_phase,ant2_phase,coord1,coord2
0,560,140,2,58,52,57,1320,1904,1,27.459060,92.070625,0.992894,-1.418147,1,1
1,560,140,2,58,52,57,1320,1904,2,27.459060,167.839209,-0.992894,1.994465,1,1
2,560,140,2,58,52,57,1320,1904,3,28.844410,94.339811,0.982794,-1.485895,1,1
3,560,140,2,58,52,57,1320,1904,4,29.154759,174.642492,-1.030377,1.983207,1,1
4,560,140,2,58,52,57,1320,1904,5,30.805844,89.453899,0.946773,-1.470015,1,1


In [8]:
features_to_scale = ['csi_len', 'payload_length', 'block_length', 'ant1_amplitude', 'ant2_amplitude', 'ant1_phase', 'ant2_phase', 'rssi', 'rssi1', 'rssi2']
scaler = StandardScaler()

# Apply standardization on features
# data = data_cleaned.copy()
data[features_to_scale] = scaler.fit_transform(data[features_to_scale])

In [9]:
data.head()

,csi_len,rate,nc,rssi,rssi1,rssi2,payload_length,block_length,subcarriers,ant1_amplitude,ant2_amplitude,ant1_phase,ant2_phase,coord1,coord2
0,0.407971,140,2,-0.721583,-0.946945,-0.504783,-1.281532,-0.799796,1,-2.330617,-0.802911,0.543963,-0.777370,1,1
1,0.407971,140,2,-0.721583,-0.946945,-0.504783,-1.281532,-0.799796,2,-2.330617,0.218152,-0.550961,1.103665,1,1
2,0.407971,140,2,-0.721583,-0.946945,-0.504783,-1.281532,-0.799796,3,-2.309207,-0.772331,0.538394,-0.814712,1,1
3,0.407971,140,2,-0.721583,-0.946945,-0.504783,-1.281532,-0.799796,4,-2.304411,0.309834,-0.571628,1.097460,1,1
4,0.407971,140,2,-0.721583,-0.946945,-0.504783,-1.281532,-0.799796,5,-2.278896,-0.838174,0.518533,-0.805960,1,1


In [10]:
data = pd.concat([data, pd.get_dummies(data['subcarriers'], prefix='subcarrier')], axis=1)
data.drop('subcarriers', axis=1, inplace=True)
data.head()

,csi_len,rate,nc,rssi,rssi1,rssi2,payload_length,block_length,ant1_amplitude,ant2_amplitude,...,subcarrier_47,subcarrier_48,subcarrier_49,subcarrier_50,subcarrier_51,subcarrier_52,subcarrier_53,subcarrier_54,subcarrier_55,subcarrier_56
0,0.407971,140,2,-0.721583,-0.946945,-0.504783,-1.281532,-0.799796,-2.330617,-0.802911,...,False,False,False,False,False,False,False,False,False,False
1,0.407971,140,2,-0.721583,-0.946945,-0.504783,-1.281532,-0.799796,-2.330617,0.218152,...,False,False,False,False,False,False,False,False,False,False
2,0.407971,140,2,-0.721583,-0.946945,-0.504783,-1.281532,-0.799796,-2.309207,-0.772331,...,False,False,False,False,False,False,False,False,False,False
3,0.407971,140,2,-0.721583,-0.946945,-0.504783,-1.281532,-0.799796,-2.304411,0.309834,...,False,False,False,False,False,False,False,False,False,False
4,0.407971,140,2,-0.721583,-0.946945,-0.504783,-1.281532,-0.799796,-2.278896,-0.838174,...,False,False,False,False,False,False,False,False,False,False


# Model

In [11]:
X = data.drop(columns=['coord1', 'coord2']) 
y = data[['coord1', 'coord2']]

In [12]:
# First, split the data into training and a temporary set (which will later be split into validation and test sets)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)  # 60% train, 40% temp

# Now, split the temporary set into validation and test sets
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # 20% val, 20% test

# Check the shapes of your splits
print("Training set shape:", X_train.shape)
print("Validation set shape:", X_val.shape)
print("Test set shape:", X_test.shape)

Training set shape: (7484064, 68)
Validation set shape: (2494688, 68)
Test set shape: (2494688, 68)


In [13]:
def create_dnn_model(input_shape):
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=input_shape))
    
    # Hidden layers
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(32, activation='relu'))
    
    # Output layer for regression (2 neurons for x and y coordinates)
    model.add(layers.Dense(2))  # For 2D localization, change to 3 for 3D localization

    # Compile the model
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

# Example of creating the model, input_shape should be (number of features,)
input_shape = (X_train.shape[1],)  # Adjust based on your number of features
model = create_dnn_model(input_shape)

# Print the model summary
model.summary()

/home/ax-ubuntu/.local/lib/python3.10/site-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
I0000 00:00:1727013454.544537    8688 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727013454.545026    8688 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727013454.545092    8688 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727013454.546057    8688 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-09-22 23:57:34.546178

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         8,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,234 (75.13 KB)

 Trainable params: 19,234 (75.13 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
# Train the model
with tf.device('/GPU:0'):
    history = model.fit(X_train, y_train, 
                        epochs=50, 
                        batch_size=4, 
                        validation_data=(X_val, y_val),
                        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)])

    # Evaluate the model on test data
    test_loss, test_mae = model.evaluate(X_test, y_test)
    print(f"Test Loss: {test_loss}, Test MAE: {test_mae}")


: 

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
